In [1]:
from sympy import *

# Téma: Náhodné vektory

## Příklad (7.1)
**Nechť náhodný vektor $(X,Y)$ má sdruženou hustotu (pravděpodobnostní funkci ;))**

| **X\Y** | **0**   | **1/2**   | **1** |
| ---     | --- | ------------- | --- |
| **1**   | 0.4 | 0.15          | 0.05|
| **2**   | 0.3 | 0.06          | 0.04|

**Najděte:**

**a) Marginální rozdělení náhodných veličin $X$ a $Y$.**

Vysčítáme řádky (pro $X$) a sloupce (pro $Y$) - doplním do tabulky:

| **X\Y** | **0**   | **1/2**   | **1** | **P(X)** |
| ---     | --- | ------------- | --- | --- |
| **1**   | 0.4 | 0.15          | 0.05| **0.6** |
| **2**   | 0.3 | 0.06          | 0.04| **0.4** |
| **P(Y)**|**0.7**| **0.21** | **0.09** | |

**b) Jsou $X$ a $Y$ nezávislé?**

Nejsou, $P(X,Y)\ne P(X)P(Y)$, a totéž **v důsledku** neplatí ani pro střední hodnoty.

**c) Spočtěte $EZ = E(X+2Y).$**

Snadno podle vzorce,

$$
EZ = E(X+2Y) = EX + 2EY =
\underbrace{1\cdot 0.6 + 2\cdot 0.4}_{EX} 
+ \underbrace{2\left(0\cdot0.7 + \frac{1}{2}\cdot0.21 + 1\cdot0.09\right)}_{2EY} = 1.79.
$$

**d) Najděte podmíněné rozdělení pro $X$ dáno $Y=y$.**

Opět snadno podle vzorce,

$$
P(X|Y=y) = \frac{P(X,Y)}{P(Y)},
$$

tedy jen podělíme tabulku hodnotami $P(Y)$,

| **X\Y** | **0**   | **1/2**   | **1** |
| ---     | --- | ------------- | --- |
| **1**   | 4/7 | 0.15/0.21=5/7 | 5/9 |
| **2**   | 3/7 | 0.06/0.21=2/7 | 4/9 |

**(Vysvětlete, jaké jsou součty řádků a sloupců a proč!)**

**e) Najděte podmíněné rozdělení $X+2Y$ dáno $Y=y$.**

Vezmeme si na pomoc předchozí tabulku a nezapomeneme místo hodnot $y$ uvažovat hodnoty $2y$. Pak jen kopírujeme čísla do příslušných buněk.

| **X+2Y\Y** | **0**   | **1/2**   | **1** |
| ---   | --- | ------------- | --- |
| **1** | 4/7 | 0 | 0 |
| **2** | 3/7 | 5/7 | 0 |
| **3** |   0 | 2/7 | 5/9 |
| **4** |  0 | 0 | 4/9 |

**f) Najděte rozdělení náhodné veličiny $Z=X+2Y$.**

## Příklad (7.2)

**Nechť náhodné veličiny $X,Y$ mají sdruženou hustotu**

$$
f(x,y) =
\begin{cases}
\frac{1}{5}(4xy + 4x - 8y) &\text{pro}\ 1\leq x \leq 2, 0\leq y \leq 1,\\
0 &\text{jinde.}
\end{cases}
$$

**a) Jsou $X$ a $Y$ nezávislé?**

Z definice, ověříme zda $f(x,y) = f(x)\cdot f(y)$.

$$
\begin{align}
f(x) &= \int_{\mathbb{R}} f(x,y) dy = \frac{1}{5}\int_0^1 4xy + 4x -8y dy 
= \frac{1}{5}(6x-4), \\
f(y) &= \int_{\mathbb{R}} f(x,y) dx = \frac{1}{5}\int_1^2 4xy -4x -8y dx =
\frac{1}{5}(6-2y).
\end{align}
$$

Nejsou zřejmě nezávislé, neboť nesplňují výše uvedenou podmínku.

In [2]:
x, y = symbols('x y')
f = 1/5 * (4*x*y + 4*x - 8*y)

fx = integrate(f, (y, 0, 1))
fy = integrate(f, (x, 1, 2))
print('f(x)=', fx, '\nf(y)=', fy)

f(x)= 1.2*x - 0.8 
f(y)= -0.4*y + 1.2


**b) Najděte $E(X+Y)$.**

Máme $f(x)$ a $f(y)$ a víme, že $EX=\int_{\mathbb{R}} x\cdot f(x)dx$, analogicky pro $EY$ a také víme, že střední hodnota je lineární zobrazení, tedy $E(X+Y) = EX + EY$. Spočteme příslušné integrály (na souvisejících intervalech) a dostaneme $\frac{31}{15}$, neboli

In [3]:
EX = integrate(x*fx, (x, 1, 2))
EY = integrate(y*fy, (y, 0, 1))
print('E(X+Y)=', EX+EY)

E(X+Y)= 2.06666666666667


**c) Najděte podmíněnou hustotu $f_{Y|X}(y|x)$.**

Opět známe už z pravděpodobnosti,

$$
f_{Y|X}(Y|X=x) = \frac{f(x,y)}{f(x)} = \frac{2xy + 2x - 4y}{3x - 2}.
$$

In [4]:
print('f(y|X=x)=', simplify(f/fx))

f(y|X=x)= (0.8*x*y + 0.8*x - 1.6*y)/(1.2*x - 0.8)


**d) Najděte $E(X|Y=y)$.**

Snadno podle

$$
E(X|Y=y) = \int_\mathbb{R} x f(x|y) dx = \frac{14-4y}{9-3y}.
$$

## Příklad (7.3)

**Nechť náhodné veličiny $X$ a $Y$ mají sdruženou hustotu**

$$
f(x,y) =
\begin{cases}
\frac{1}{2} y e^{-(x-y^2)/2} &\text{pro}\ x,y>0,\\
0 &\text{jinde.}
\end{cases}
$$

**a) Jsou $X$ a $Y$ nezávislé?**

Spočteme příslušné marginální hustoty:

$$
\begin{align}
f_X(x) &= \int_{\mathbb{R}} f(x,y) dy = \int_0^\infty \frac{1}{2} y e^{(-x-y^2)/2}dy = \frac{1}{2} e^{-\frac{x}{2}}\int_0^\infty ye^{-\frac{y}{2}} dy \\ &= |\text{substitucí}| = \frac{1}{2}e^{-\frac{x}{2}} \cdot \left[ -e^{-\frac{y^2}{2}} \right]_0^\infty = \frac{1}{2}e^{-\frac{x}{2}},
\\
f_Y(y) &= \int_{\mathbb{R}} f(x,y) dx = \frac{1}{2}ye^{-\frac{y^2}{2}} \int_0^\infty e^{-\frac{x}{2}}dx = \frac{1}{2}ye^{-\frac{y^2}{2}}\cdot \left[-2e^{-\frac{x}{2}} \right]_0^\infty = ye^{-\frac{y^2}{2}}.
\end{align}
$$

Pronásobením vidíme, že $f_X(x)f_Y(y) = f(x,y)$, veličiny jsou tedy nezávislé. V `sympy`:

In [14]:
fxy = 1/2 * y * exp((-x-y**2)/2)
fx = integrate(fxy, (y, 0, oo))
fy = integrate(fxy, (x, 0, oo))
print('f(x) = ', fx)
print('f(y) = ', fy)
print('f(x)f(y) = ', fx*fy)
print('f(x)f(y) = f(x,y)', simplify(fx*fy)==simplify(fxy))

f(x) =  0.5*exp(-x/2)
f(y) =  1.0*y*exp(-y**2/2)
f(x)f(y) =  0.5*y*exp(-x/2)*exp(-y**2/2)
f(x)f(y) = f(x,y) True


**b) Najděte $E(X+Y)$.**

Snadno z definice $E(X+Y) = EX + EY$ a z informace, že 

$$
\int_0^\infty y^2 e^{-ay^2}dy = \frac{\sqrt{\pi}}{4a^{\frac{3}{2}}}.
$$

Tedy

$$
\begin{align}
EX &= \int_\mathbb{R} x f_X(x) dx = \int_0^\infty x \cdot \frac{1}{2}e^{-\frac{x}{2}} dx \\ &= \frac{1}{2} \int_0^\infty x\cdot  e^{-\frac{x}{2}}dx = |\text{per partes, $u=x, v'=e^{-x/2}$}| = 2,\\
EY &= \int_\mathbb{R} y\cdot f_Y(y) dy = \frac{\sqrt{2\pi}}{2}.
\end{align}
$$

Pro střední hodnotu součtu tedy dostaneme

$$
E(X+Y) = EX + EY = 2+\frac{\sqrt{2\pi}}{2}.
$$

In [29]:
EX = integrate(x*fx, (x, 0, oo))
EY = integrate(y*fy, (y, 0, oo))
print('EX = ', EX)
print('EY = ', EY)
print('EX + EY = ', simplify(EX + EY))

EX =  2.00000000000000
EY =  0.5*sqrt(2)*sqrt(pi)
EX + EY =  0.5*sqrt(2)*sqrt(pi) + 2.0


**c) Najděte $E(XY)$.**

Zde využijeme zjištěnou nezávislost $X\mathbb{\perp} Y$, neboť tehdy platí, že $E(XY) = EX \cdot EY$, z čehož rovnou plyne výsledek $\sqrt{2\pi}$.

## Příklad (7.4)

**Nechť náhodné veličiny $X$ a $Y$ mají sdruženou hustotu**

$$
f(x,y) =
\begin{cases}
8xy &\text{pro}\ 0\leq x\leq 1, 0\leq y \leq x, \\
0   &\text{jinde}.
\end{cases}
$$

**a) Jsou $X$ a $Y$ nezávislé?**

Závislost celkem patrně plyne z definice $f(x,y)$, ověříme snadno výpočtem příslušných marginálních hustot, stejným způsobem, jako výše.

**b) Najděte marginální hustotu $X$.**

$$
f_X(x) = \int_\mathbb{R} f(x,y)dy = \int_0^x 8xy dx = 4x^3.
$$

**c) Určete $EX$**

$$
EX = \int_\mathbb{R} x\cdot f_X(x) dx = \int_0^1 4x^4dx = \frac{4}{5}.
$$

## Příklad (7.5)
**Nechť náhodné veličiny $X$ a $Y$ mají sdruženou hustotu**

$$
f(x,y) =
\begin{cases}
\frac{1}{3}(-4xy - 4x +8y +8) &\text{pro} x\in[1,2], y\in[0,1],\\
0   &\text{jinde}.
\end{cases}
$$

**a) Jsou $X$ a $Y$ nezávislé?**

Stačí spočítat marginální hustoty a ověřit $f(x,y) = f_X(x)\cdot f_Y(y)$. Příklad je jednoduchý a odpověď je *ano, jsou nezávislé*.

In [44]:
fxy = 1/3 * (-4*x*y - 4*x + 8*y + 8)
fx = integrate(fxy, (y, 0, 1))
fy = integrate(fxy, (x, 1, 2))
print('f(x) = ', fx)
print('f(y) = ', fy)

f(x) =  -2.0*x + 4.0
f(y) =  0.666666666666667*y + 0.666666666666667


**b) Najděte $E(X+Y)$**

$$
E(X+Y) = EX + EY = \frac{17}{9}.
$$

In [47]:
EX = integrate(x*fx, (x, 1, 2))
EY = integrate(y*fy, (y, 0, 1))
print('EX = ', EX)
print('EY = ', EY)
print('E(X+Y) = EX + EY = ', EX + EY)

EX =  1.33333333333333
EY =  0.555555555555555
E(X+Y) = EX + EY =  1.88888888888889


**c) Najděte $E(XY)$**

Z $X\perp Y$ plyne $E(XY) = EX\cdot EY = \frac{20}{27}$.

In [50]:
print('EX * EY = ', EX*EY)

EX * EY =  0.740740740740741


## Příklad (7.6)
**Nechť náhodné veličiny $X$ a $Y$ mají sdruženou hustotu**

$$
f(x,y) =
\begin{cases}
\frac{3}{8} xy^2 &\text{pro}\ x\in[0,1], y\in[-2,2],\\
0   &\text{jinde}.
\end{cases}
$$

**a) Najděte marginální hustotu veličiny $X$.**

Snadno vyintegrováním $y$,

$$
f_X(x) = \int_\mathbb{R} f(x,y) dy = 2x.
$$

**b) Najděte podmíněnou hustotu $f_{Y|X}(y|x)$.**

Snadno z definice

$$
f_{Y|X}(y|x) = \frac{f(x,y)}{f_X(x)} = \frac{3}{16}y^2.
$$

## Příklad (7.7)

**Nechť $X_1$ a $X2$ jsou dvě nezávislé veličiny se střední hodnotou $\mu$ a rozptylem $\sigma^2$.**

**a) Najděte střední hodnotu a rozptyl náhodných veličin v tabulce:**

| veličina | $EY_i$ | $\operatorname{var}Y_i$ |
|---|---|---|
|$Y_1 = X_1+X_2$ | ? | ? |
|$Y_2 = 2X_1$| ? | ? |
|$Y_3 = X_1-X_2$| ? | ? |

Řešení - mysleme na transformace náhodných veličin, viz minulé cvičení!

| veličina | $EY_i$ | $\operatorname{var}Y_i$ |
|---|---|---|
|$Y_1 = X_1+X_2$ | $2\mu$ | $2\sigma^2$|
|$Y_2 = 2X_1$| $2\mu$ | $4\sigma^2$ |
|$Y_3 = X_1-X_2$| 0 | $2\sigma^2$ |


**b) Spočtěte kovarianci všech dvojic veličin $Y_i$**

Kovariance najdeme v kovarianční matici náhodného vektoru $Y=[Y_1, Y_2, Y_3]$. Připomeňme, že $\operatorname{cov}(A,B) = \operatorname{cov}(B,A)$, matice bude tedy symetrická a na diagonále nalezneme variance veličin, neboť $\operatorname{cov}(A,A) = \operatorname{var}A$.

$$
\operatorname{cov}(Y) =
\begin{bmatrix}
2\sigma^2 & 2\sigma^2 & 0\\
2\sigma^2 & 4\sigma^2 & 2\sigma^2 \\
0 & 2\sigma^2 & 2\sigma^2
\end{bmatrix}
$$


**c) Co lze říci o nezávislosti veličin $Y_1, Y_2, Y_3$?**

Pohledem na definice veličin:

- $Y_1, Y_2$ nejsou nezávislé
- $Y_2, Y_3$ nejsou nezávislé
- $Y_1, Y_3$ nelze rozhodnout

## Příklad (7.8)
**Nechť $X$ a $Y$ jsou nezávislé náhodné veličiny s geometrickým rozdělením s parametrem $p$. Určete:**

**a) Rozdělení $\min(X, Y)$,**

Připomeňme, že distribuční funkce geom. rozdělení s parametrem $p$ je $P(X\leq k) = 1-(1-p)^k$.

$$
P(\min(X,Y)\leq k) = 1-P(\min(X,Y) > k) = 1 - P(\{X>k\}\cap \{Y>k\})
= 1-P(X>k)\cdot P(Y>k) = 1-(1-p)^k\cdot (1-p)^k = 1-[(1-p)^2]^k,
$$

což je distribuční funkce geometrického rozdělení s parametrem $1-(1-p)^2$.


**b) $E(\max\{X,Y\})$,**

Protože 

$$
\max(X,Y) = X + Y - \min(X,Y),
$$

máme s využitím výsledku z bodu (a)

$$
E(\max(X,Y)) = EX + EY - E[\min(X,Y)] = \frac{1}{p} + \frac{1}{p} - \frac{1}{1-(1-p)^2}
$$

**c) rozdělení $\max\{X,Y\}$,**

tbd

**d) $P(X<Y)$.**

Jelikož 

$$1 = P(X<Y) + P(X=Y) + P(Y<X) = 2P(X<Y) + P(X=Y),$$

dostaneme

$$
\begin{align}
P(X<Y) &= \frac{1-P(X=Y)}{2} = \frac{1-\sum_{k=0}^\infty P(X=k)P(Y=k)}{2}
\\ &
= \frac{1-\sum_{k=0}^\infty [p(1-p)k]^2}{2} = \frac{1+\frac{p}{p-2}}{2}.
\end{align}
$$
---

Pozn.: užitečná může být obecně následující formulka - [Tail-sum formula](https://en.wikipedia.org/wiki/Expected_value#Alternative_formula_for_expected_value) pro diskrétní veličinu s hodnotami v $\{1,2,\ldots\}$ říká, že

$$
EX = \sum_{k=1}^\infty P(X> k).
$$

